In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import metrics 
import matplotlib.pyplot as plt
import seaborn as sns

**İleri aşamalarda kullanılacak veri setleri yükleniyor.**

In [ ]:
submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
sales = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

* Günlük satılan ürün sayısına göre sınırlar tespit ediliyor.
* Günlük satılan ürünlerin fiyatlarına göre sınırlar tespit ediliyor.

In [ ]:
plt.figure(figsize=(10,4))
plt.xlim(sales.item_cnt_day.min(), sales.item_cnt_day.max()*1.1)
sns.boxplot(x=sales.item_cnt_day)

plt.figure(figsize=(10,4))
plt.xlim(sales.item_price.min(), sales.item_price.max()*1.1)
sns.boxplot(x=sales.item_price)

**Yukarıda incelenen sonuçlara göre preprocessing işlemleri uygulanıyor.**

In [ ]:
# Günlük satış sayısı 0 dan düşük olanlar veri setinden çıkarılıyor.
sales.drop(sales[sales['item_cnt_day'] < 0].index, axis=0, inplace=True)

# Tekrar eden veriler veri setinden çıkartılıyor.
sales.drop_duplicates(subset=['date', 'date_block_num', 'shop_id', 'item_id', 'item_cnt_day'] , inplace=True)

# Yukarıdaki grafiklerden yola çıkarak sınırlar belirleniyor.
sales = sales[sales.item_cnt_day < 1200]
sales = sales[sales.item_price < 100000]


**Preprocessing işleminden sonra grafikleri tekrar çizdirerek yapılan işlemleri görebiliyoruz.**

In [ ]:
plt.figure(figsize=(10,4))
plt.xlim(sales.item_cnt_day.min() - 50, sales.item_cnt_day.max()*1.1)
sns.boxplot(x=sales.item_cnt_day)

plt.figure(figsize=(10,4))
plt.xlim(sales.item_price.min() - 500, sales.item_price.max()*1.1)
sns.boxplot(x=sales.item_price)

In [ ]:
agg_df = sales.groupby(["date_block_num", "shop_id", "item_id"])["item_cnt_day"].agg('sum').reset_index()
agg_df.columns = ['date_block_num', 'shop_id','item_id', 'item_cnt_day']
agg_df['item_cnt_day'].clip(0, 20, inplace=True)

features = agg_df.iloc[:,:-1]
itemCount = agg_df.iloc[:,-1:]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, itemCount, test_size=0.2, random_state=1)

In [ ]:
rf = RandomForestRegressor(n_estimators = 50, random_state = 1, verbose = 1)
rf.fit(x_train, y_train)

y_pred = rf.predict(x_test)

rf_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

rf_r2 = r2_score(y_test,y_pred)

print("R2 Score:", r2_score(y_test, y_pred))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Root Mean Squared Error :', np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
t_p = rf.predict(test)
p_df = pd.DataFrame(t_p, columns=['item_cnt_month'])
p_df = p_df.clip(0,20)
submission.drop(columns = "item_cnt_month", inplace = True)
result = pd.concat([submission,p_df], axis=1)
result.head(10)

In [ ]:
result.to_csv('submission.csv', index=False)